# Step 5 - SSE Consumer

Consume the WMF Stream and puts each message into our SQS queue

In [ ]:
import boto3
import json
from sseclient import SSEClient as EventSource

#SQS client library
sqs = boto3.client('sqs'
    , endpoint_url="http://localhost:4576" #only for test purposes
    , use_ssl=False #only for test purposes
    , region_name='us-east-1')
queue_url = 'http://localhost:4576/queue/sse_queue'

def catch_events():
    url = 'https://stream.wikimedia.org/v2/stream/recentchange'
    for event in EventSource(url):
        if event.event == 'message':
            try:
                message = json.loads(event.data)
            except ValueError:
                pass
            else: 
                enqueue_message( json.dumps(message) )

def enqueue_message( message ):
    response = sqs.send_message(
        QueueUrl = queue_url,
        DelaySeconds=1,
        MessageBody = message
    )
    print('\rMessage %s enqueued' % response['MessageId'], sep=' ', end='', flush=True)
  

In [ ]:
catch_events()